# DEXES memecoins

## Importaciones

In [ ]:
from solders.keypair import Keypair
from typing import Literal, Union

from solana_manager import SolanaAccountInfo, SolanaWalletManager
from pumpfun import PumpFunTransactions, PumpFunPriceFetcher
from dexscreener import DexScreenerPriceTracker, DexScreenerPortfolioMonitor

## Configuraciones

In [ ]:
NETWORK = "mainnet-beta"
RPC_URL = "https://api.mainnet-beta.solana.com"
WALLET_FILE = "wallets/wallet_pumpfun.json"
SLIPPAGE = 15.0
PRIORITY_FEE = 0.0001

wallet_data = None
async with SolanaWalletManager(network=NETWORK) as solana_manager:
    wallet_data = await solana_manager.load_wallet_from_file(filename=WALLET_FILE)
    wallet_data["key_pair"] = Keypair().from_base58_string(wallet_data["private_key"])

account_info = SolanaAccountInfo(network=NETWORK)
transactions = PumpFunTransactions()
pump_fetcher = PumpFunPriceFetcher()
price_tracker = DexScreenerPriceTracker()

if not wallet_data:
    raise Exception("Wallet not found")

portfolio_monitor = DexScreenerPortfolioMonitor(wallet_data.get("public_key"), price_tracker)

## Información

### Balance

In [ ]:
async with portfolio_monitor:
    await portfolio_monitor.get_detailed_balance()


# Trading

In [ ]:
TOKEN_MINT="6F6jjd71nbjwUZNi93wpXimbyH12CLkeMNJ6X1brpump"

### Precio de token

In [ ]:
token_price = None
async with price_tracker:
    token_price = await price_tracker.get_token_price(TOKEN_MINT)

if not token_price:
    print("🔄 Reintentando con Pump.fun Price Fetcher")
    token_price = None
    async with pump_fetcher:
        token_price = await pump_fetcher.get_token_price(TOKEN_MINT)

if token_price:
    print(f"\n🎉 PRECIO OBTENIDO EXITOSAMENTE!")
    print(f"💰 Precio SOL: {token_price.price_sol:.12f}")
    print(f"💵 Precio USD: ${token_price.price_usd:.12f}")
    if hasattr(token_price, 'market_cap_usd'):
        print(f"📊 Market Cap: ${token_price.market_cap_usd:,.2f}")
    if hasattr(token_price, 'bonding_progress'):
        print(f"📈 Progreso Bonding: {token_price.bonding_progress:.2f}%")
    print(f"⏰ Timestamp: {token_price.timestamp}")
else:
    print("❌ No se pudo obtener el precio")

### Local trade

In [ ]:
# Mints: 6F6jjd71nbjwUZNi93wpXimbyH12CLkeMNJ6X1brpump, CSDX82AZb8DTCMUMRDaJ5nRXPvvm2RNAf7NgHxhspump, ExSqaSHmVcn3EFSRjS3uVsZezkU1FZkJAjamNt2Npump
TOKEN_MINT="6F6jjd71nbjwUZNi93wpXimbyH12CLkeMNJ6X1brpump"
ACTION: Literal["buy", "sell"] = "sell"
AMOUNT: Union[float, str] = "0.000001"
DENOMINATED_IN_SOL: bool = True
SLIPPAGE = "15.0"
PRIORITY_FEE = "0.000001"

async with transactions:
    signature = await transactions.create_and_send_local_trade(
        keypair=wallet_data.get("key_pair"), # type: ignore
        action=ACTION,
        mint=TOKEN_MINT,
        amount=AMOUNT,
        denominated_in_sol=DENOMINATED_IN_SOL,
        slippage=SLIPPAGE,
        priority_fee=PRIORITY_FEE
    )
    print(f"🔗 Transacción enviada exitosamente. Puedes verla en Solscan: https://solscan.io/tx/{signature}?cluster=mainnet-beta")